In [4]:
!pip install unsloth

  Using cached unsloth-2025.9.1-py3-none-any.whl (313 kB)
  Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Using cached trl-0.22.2-py3-none-any.whl (544 kB)
  Using cached hf_transfer-0.1.9-cp38-abi3-macosx_11_0_arm64.whl (1.3 MB)
  Using cached sentencepiece-0.2.1-cp39-cp39-macosx_11_0_arm64.whl (1.3 MB)
  Using cached accelerate-1.10.1-py3-none-any.whl (374 kB)
  Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
  Using cached xformers-0.0.32.post2.tar.gz (12.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached bitsandbytes-0.42.0-py3-none-any.whl (105.0 MB)
  Using cached protobuf-6.32.0-cp39-abi3-macosx_10_9_universal2.whl (426 kB)
  Using cached diffusers-0.35.1-py3-none-any.whl (4.1 MB)
  Using cached tyro-0.9.31-py3-none-any.whl (131 kB)
  Using cached unsloth_zoo-2025.9.2-py3-none-any.whl (197 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached huggingface_

In [1]:
import json

# Load your dataset
with open(r"/Users/zudu/Downloads/Abu/Data/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

alpaca_data = []

for row in data:
    instruction = row["instruction"]
    output = row["output"]

    # Tamil input
    if row.get("input_ta"):
        alpaca_data.append({
            "instruction": instruction,
            "input": row["input_ta"],
            "output": output
        })
    
    # English input
    if row.get("input_en"):
        alpaca_data.append({
            "instruction": instruction,
            "input": row["input_en"],
            "output": output
        })

# Save in Alpaca format
with open("tanglish_alpaca.json", "w", encoding="utf-8") as f:
    json.dump(alpaca_data, f, ensure_ascii=False, indent=2)

print(f"Converted {len(alpaca_data)} examples into Alpaca format!")


Converted 14986 examples into Alpaca format!


In [8]:
!pip install -U sarvamai

     |████████████████████████████████| 167 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 19.3 MB/s eta 0:00:01
     |████████████████████████████████| 444 kB 29.5 MB/s eta 0:00:01
  Using cached idna-3.10-py3-none-any.whl (70 kB)
     |████████████████████████████████| 107 kB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 21.9 MB/s eta 0:00:01
  Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
     |████████████████████████████████| 1.9 MB 37.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/zudu/Downloads/Abu/fine/bin/python3 -m pip install --upgrade pip' command.


In [9]:
from sarvamai import SarvamAI

client = SarvamAI(
    api_subscription_key="sk_0nv4bdy3_4vqngfJKql3fopwyLWW1v0XF",
)

response = client.text.translate(
    input="Hi, My Name is Vinayak.",
    source_language_code="auto",
    target_language_code="gu-IN",
    speaker_gender="Male"
)

print(response)


request_id='20250906_f0a1c5e1-8128-4551-93ff-6cbda0a0938a' translated_text='નમસ્તે, મારું નામ વિનાયક છે.' source_language_code='en-IN'


In [16]:
!pip install nest-asyncio

You should consider upgrading via the '/Users/zudu/Downloads/Abu/fine/bin/python3 -m pip install --upgrade pip' command.


In [15]:
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp39-cp39-macosx_10_9_universal2.whl size=39131 sha256=e3450f6f1944bc824c1be2c368b39997930bfff8ab262005d89a9cc78c01eb71
  Stored in directory: /Users/zudu/Library/Caches/pip/wheels/28/d3/62/6ad369dc09fe82e1c9ceb83601a800eb305b901df7789aa550
Successfully built pyaudio
You should consider upgrading via the '/Users/zudu/Downloads/Abu/fine/bin/python3 -m pip install --upgrade pip' command.


In [41]:
import asyncio
import base64
import nest_asyncio
import pyaudio
import wave
import io
import json
from sarvamai import AsyncSarvamAI

nest_asyncio.apply()

# Audio recording constants
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000

def pcm_to_wav_bytes(pcm_bytes):
    """Wrap raw PCM into a minimal WAV header in memory."""
    buffer = io.BytesIO()
    with wave.open(buffer, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(2)   # paInt16 = 2 bytes
        wf.setframerate(RATE)
        wf.writeframes(pcm_bytes)
    return buffer.getvalue()

async def real_time_transcribe():
    client = AsyncSarvamAI(api_subscription_key="sk_0nv4bdy3_4vqngfJKql3fopwyLWW1v0XF")
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                    input=True, frames_per_buffer=CHUNK)

    try:
        async with client.speech_to_text_streaming.connect(
            language_code="gu-IN",
            model="saarika:v2.5",
            high_vad_sensitivity=True,
            vad_signals=True
        ) as ws:
            print("Started listening... Press Ctrl+C to stop")

            while True:
                pcm = stream.read(CHUNK, exception_on_overflow=False)
                wav_bytes = pcm_to_wav_bytes(pcm)
                b64data = base64.b64encode(wav_bytes).decode("utf-8")

                # Send WAV audio
                await ws.transcribe(audio=b64data, encoding="audio/wav", sample_rate=RATE)

                # Receive server response
                raw = await ws.recv()
                try:
                    message = json.loads(raw) if isinstance(raw, str) else raw
                except Exception:
                    print("Raw:", raw)
                    continue

                if message.get("type") == "transcript":
                    print(f"Transcription: {message.get('text')}")
                elif message.get("type") == "speech_start":
                    print("Speech detected")
                elif message.get("type") == "speech_end":
                    print("Speech ended")
                else:
                    print("Other:", message)

    except KeyboardInterrupt:
        print("\nStopping...")
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()

if __name__ == "__main__":
    asyncio.run(real_time_transcribe())


||PaMacCore (AUHAL)|| Warning on line 521: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Warning on line 441: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Error on line 1322: err='-10851', msg=Audio Unit: Invalid Property Value


OSError: [Errno -9986] Internal PortAudio error

In [40]:
import pyaudio

p = pyaudio.PyAudio()

print("Available audio devices:")
for i in range(p.get_device_count()):
    info = p.get_device_info_by_index(i)
    print(f"{i}: {info['name']} (inputs: {info['maxInputChannels']})")

p.terminate()


Available audio devices:
0: OnePlus Buds 3 (inputs: 1)
1: OnePlus Buds 3 (inputs: 0)
2: MacBook Air Microphone (inputs: 1)
3: MacBook Air Speakers (inputs: 0)


In [44]:
!pip install sounddevice numpy

  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
You should consider upgrading via the '/Users/zudu/Downloads/Abu/fine/bin/python3 -m pip install --upgrade pip' command.


In [45]:
import sounddevice as sd
import numpy as np

CHUNK = 1024
RATE = 16000
CHANNELS = 1

def callback(indata, frames, time, status):
    if status:
        print(status)
    audio_data = indata.copy().tobytes()
    # here you can send audio_data to SarvamAI API

stream = sd.InputStream(
    channels=CHANNELS,
    samplerate=RATE,
    blocksize=CHUNK,
    dtype='int16',
    callback=callback
)

with stream:
    print("Listening... Press Ctrl+C to stop")
    while True:
        pass


Listening... Press Ctrl+C to stop


KeyboardInterrupt: 

In [39]:
!pip install --no-binary :all: pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp39-cp39-macosx_10_9_universal2.whl size=39131 sha256=4b16e42dd75be3d59ca761155beaab440402c787f9c104274618e0d6b87b5386
  Stored in directory: /Users/zudu/Library/Caches/pip/wheels/28/d3/62/6ad369dc09fe82e1c9ceb83601a800eb305b901df7789aa550
Successfully built pyaudio
You should consider upgrading via the '/Users/zudu/Downloads/Abu/fine/bin/python3 -m pip install --upgrade pip' command.


In [47]:
import asyncio
import sounddevice as sd
import numpy as np
import base64
from sarvamai import AsyncSarvamAI

# Audio recording parameters
CHUNK = 1024
RATE = 16000
CHANNELS = 1
DTYPE = 'int16'

API_KEY = "sk_0nv4bdy3_4vqngfJKql3fopwyLWW1v0XF"   # replace with your key

async def real_time_transcribe():
    client = AsyncSarvamAI(api_subscription_key=API_KEY)

    async with client.speech_to_text_streaming.connect(
        language_code="gu-IN",
        model="saarika:v2.5",
        high_vad_sensitivity=True,
        vad_signals=True
    ) as ws:
        print("🎤 Started listening... Press Ctrl+C to stop")

        loop = asyncio.get_event_loop()
        queue = asyncio.Queue()

        def callback(indata, frames, time, status):
            if status:
                print("⚠️", status)
            loop.call_soon_threadsafe(queue.put_nowait, indata.copy())

        with sd.InputStream(
            channels=CHANNELS,
            samplerate=RATE,
            blocksize=CHUNK,
            dtype=DTYPE,
            callback=callback
        ):
            while True:
                indata = await queue.get()
                audio_bytes = indata.tobytes()
                b64data = base64.b64encode(audio_bytes).decode("utf-8")

                # ✅ Use SDK helper instead of raw .send()
                await ws.transcribe(audio=b64data)

                # ✅ Receive server response
                message = await ws.receive_json()
                if message.get("type") == "transcript":
                    print("📝 Transcription:", message.get("text"))
                elif message.get("type") == "speech_start":
                    print("🎙️ Speech detected")
                elif message.get("type") == "speech_end":
                    print("🔇 Speech ended")

if __name__ == "__main__":
    try:
        asyncio.run(real_time_transcribe())
    except KeyboardInterrupt:
        print("\n👋 Exiting...")


🎤 Started listening... Press Ctrl+C to stop


AttributeError: 'AsyncSpeechToTextStreamingSocketClient' object has no attribute 'receive_json'